## Watershed Terrain from AWS
Get a watershed terrain model for a given Lat/Long by utilizing the AWS Terrain Tiles and the EPA WATER's service

 - Created by:   __Andy Carter, PE__
 >andy.carter@austin.utexas.edu

 - Created On:   __29 Jan 2021__<br>
 - Last revised:  __Verison 0.1__
 
 - Purpose:
 >Input a Lat/Long point.  Using the EPA WATER's service, get the receiving COMID stream and then the total watershed polygon down to that stream.  With this polygon, download all the AWS tile that are needed.  Merge the dem tiles and clip to the buffered watershed boundary.

 - Inputs required:
  > 1) Lat/Long of the requested point <br>
    2) Zoom level of the requested terrain tiles (highest resolution is 14) <br>
    
 - Output generated:
  > A merged GeoTIFF dem with a COMID name called {COMID}_dem.clip.tif

***
### 1.0 References

In [ ]:
import requests
import json
import geopandas as gpd

from owslib.wms import WebMapService

import urllib
import math
import matplotlib.pyplot as plt
import os
import glob
import rasterio
import gdal
import osr

from shapely.geometry import shape

from rasterio.merge import merge
from rasterio.plot import show
from rasterio.mask import mask

***
### 2.0 Input 

In [ ]:
#Input
fltLat = 30.502
fltLong = -97.910

int_Zoom = 14 #the most detailed AWS Terrain level available

#Directory to write DEM data
strPathDirectory = r'AWS'

***
### 3.0 Get Watershed Limits from EPA WATER's web services

In [ ]:
#EPA Waters Point indexing service - determine the COMID that the selected point drains
str_PointIndexURL = r'https://ofmpub.epa.gov/waters10/PointIndexing.Service?pGeometry=POINT('
str_PointIndexURL += str(fltLong) + " "
str_PointIndexURL += str(fltLat) + ")"

#Webrequest to get json data (Point Indexing to get COMID)
r_json = requests.get(str_PointIndexURL).json()
data = r_json["output"]["ary_flowlines"]
intCOMID = data[0].get('comid')
#print("COMID: " + str(intCOMID))

#EPA Waters request for watershed - get the polygon of the drainage area including the receiving drainage COMID's catchment
str_WatershedURL = r'http://ofmpub.epa.gov/waters10/NavigationDelineation.Service?pNavigationType=UT&pStartComid='
str_WatershedURL += str(intCOMID)
str_WatershedURL += r'&pOutputFlag=FEATURE&pAggregationFlag=TRUE'

#Parse out the polygon boundaries
r_json = requests.get(str_WatershedURL).json()
data = r_json["output"]["shape"]
data = json.dumps(data) #dump to get proper json format with double quotes
df1 = gpd.read_file(data)  #geopands of the watershed polygon

#watershed_data = GeoData(geo_dataframe = df1,
#                         style={'color':'red','fillOpacity':0},
#                         name = 'Watershed')


In [ ]:
#Create an offset buffer to the watershed 
df2 = df1.buffer(0.002,resolution=8) #offsets 0.002 degrees - about 600 feet

#Show bounding envelope(green) and watershed(red)
boundary=df1.boundary.plot(color='red')
df1.plot
#df2.plot(ax=boundary, color='green');

***
### 4.0 Create URL of the tiles to reqest from AWS Terrain Tile service

In [ ]:
#get the lower left and upper right coordinates
flt_minLong = df2[0].bounds[0]
flt_minLat = df2[0].bounds[1]

flt_maxLong = df2[0].bounds[2]
flt_maxLat = df2[0].bounds[3]

In [ ]:
# https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames
def fn_deg2num(lat_deg, lon_deg, zoom):
    
    """Return the tile number (x,y) given the Lat/Long and zoom level

    Args:
        lat_deg: Point latitude in decimal degrees
        lon_deg: Point longitude in decimal degrees
        zoom: Tile pyramid zoom-in level 

    Returns:
        Integers of the x/y of the tile
    """
    
    lat_rad = math.radians(lat_deg)
    n = 2.0 ** zoom
    xtile = int((lon_deg + 180.0) / 360.0 * n)
    ytile = int((1.0 - math.asinh(math.tan(lat_rad)) / math.pi) / 2.0 * n)
    return (xtile, ytile)

In [ ]:
#indecies of tiles from within the bounding box
#print(fn_deg2num(flt_minLat,flt_minLong,int_Zoom))
#print(fn_deg2num(flt_maxLat,flt_maxLong,int_Zoom))

int_xMinTile = fn_deg2num(flt_minLat,flt_minLong,int_Zoom)[0]
int_xMaxTile = fn_deg2num(flt_maxLat,flt_maxLong,int_Zoom)[0]

int_yMinTile = fn_deg2num(flt_minLat,flt_minLong,int_Zoom)[1]
int_yMaxTile = fn_deg2num(flt_maxLat,flt_maxLong,int_Zoom)[1]

In [ ]:
#Create URL list of all the needed tiles
lst_terrainPaths = []
for x in range(int_xMinTile, int_xMaxTile+1):
    for y in range (int_yMaxTile, int_yMinTile+1):

        #strURL = r's3://elevation-tiles-prod/geotiff/'
        strURL = r'https://s3.amazonaws.com/elevation-tiles-prod/geotiff/'
        strURL += str(int_Zoom) + "/"
        strURL += str(x) + "/"
        strURL += str(y) + ".tif"
        lst_terrainPaths.append(strURL)
print("Number of tiles (Zoom Level " + str(int_Zoom) + "): " + str(len(lst_terrainPaths)))

***
### 5.0 Download the requested tiles

Can we speed this up with Dask to download multiple tiles (or glob multiple URLs) at once?

In [ ]:
%%time
lst_FileNamesToMerge = []
intCount = 1
for i in lst_terrainPaths:
    strFileName = "_" + str(intCOMID) + '_TileImage_' + str(intCount) + '.tif'
    strFilePath = strPathDirectory + strFileName
    lst_FileNamesToMerge.append(strFilePath)
    strTotalPath = strPathDirectory + strFileName
    r = requests.get(i)
    with open(strTotalPath, 'wb') as f:
        f.write(r.content)
    intCount += 1

***
### 6.0 Merge the terrain tiles

In [ ]:
#Merge the DEMs in the lst_FileNamesToMerge list
strOutTiffPath = str(intCOMID) + "_dem_merge.tif"

d = []
for file in lst_FileNamesToMerge:
    src = rasterio.open(file)
    d.append(src)
    
out_meta = src.meta.copy()

mosaic, out_trans = merge(d)

# Create Metadata of the for the mosaic TIFF
out_meta.update({"driver": "HFA","height":mosaic.shape[1],"width":mosaic.shape[2],"transform": out_trans,})

# Write the updated DEM to the specified file path
with rasterio.open(strOutTiffPath, "w", **out_meta) as dest:
    dest.write(mosaic)
    dest.close()

***
### 7.0 Clip the terrain tiles

In [ ]:
def getFeatures(gdf,int_polyIndex):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][int_polyIndex]['geometry']]

In [ ]:
# Read the overall Terrain raster
src = rasterio.open(strOutTiffPath)

# Copy the metadata of the src terrain data
out_meta = src.meta.copy()

#Get the projection of the raster
d = gdal.Open(strOutTiffPath)
proj = osr.SpatialReference(wkt=d.GetProjection())
str_EPSG_Raster = proj.GetAttrValue('AUTHORITY',1)
print(str_EPSG_Raster)

strWatershedShape_Path = str(intCOMID) + '_watershed_ar.shp'
#write the shapefile of watershed
#df1.to_file(strWatershedShape_Path)

strClip_Path = str(intCOMID) + '_dem_clip.tif'

# re-project the clip polygon into same CRS as terrain raster
#df1 = df1.to_crs(epsg=str_EPSG_Raster)
df2 = df2.to_crs(epsg=3857)

# Converts the buffer to a GeoJson version for rasterio
# currently requests the first polygon in the geometry
coords = getFeatures(df2,0)

# Clip the raster with Polygon
out_img, out_transform = mask(dataset=src, shapes=coords, crop=True)

# Metadata for the clipped image
# This uses the out_image height and width
out_meta.update({"driver": "GTiff","height":out_img.shape[1],"width":out_img.shape[2],"transform": out_transform,})

with rasterio.open(strClip_Path, "w", **out_meta) as dest:
    dest.write(out_img)

***
### 8.0 View Output

In [ ]:
src2 = rasterio.open(strClip_Path)

#print the raster
fig, ax = plt.subplots(figsize=(8, 8))
show(src2, cmap='terrain', ax=ax)

***
### 9.0 Delete the downloaded tiles

In [ ]:
#removing existing files from working folder - if necessary
intCount = 1
for i in lst_terrainPaths:
    strFileName = "_" + str(intCOMID) + '_TileImage_' + str(intCount) + '.tif'
    strFilePath =  strPathDirectory + strFileName
    if os.path.exists(strFilePath):
        os.remove(strFilePath)
    intCount += 1

In [ ]:
if os.path.exists(strOutTiffPath):
    os.remove(strOutTiffPath)